## ICSI corpus. 
Go through the list of files and extract passage and the summaries, using the DialogueActs as intermediary

In [22]:
import glob, os
import xml.etree.ElementTree as et
from collections import OrderedDict

### Extract words

In [31]:
meeting_dict = dict() # key = meeting, value = ordered dict of word id: word
for word_f in sorted(glob.glob("/home/rajiv_nair/266_project/ICSIplus/Words/*xml")):
    meeting_name = word_f.split(".")[0]
    root = et.parse(word_f).getroot()
    words = root.findall('w')
    meet_words = OrderedDict()
    for word in words:
        meet_words[word.attrib['{http://nite.sourceforge.net/}id']] = word.text
    meeting_dict[meeting_name] = meet_words

len(meeting_dict.keys())


75

### Extract dialogues

In [71]:
meeting_dialogues = dict() # key = dialogue id, value = list of words
for dialog_f in sorted(glob.glob("/home/rajiv_nair/266_project/ICSIplus/DialogueActs/*acts.xml")):
    print(dialog_f)
    root = et.parse(dialog_f).getroot()
    dlist = root.findall('dialogueact/{http://nite.sourceforge.net/}child')
    for dl in dlist:
        words = dl.attrib['href']
        start, end = words.split("..")
    break

/home/rajiv_nair/266_project/ICSIplus/DialogueActs/Bdb001.A.dialogue-acts.xml
Bdb001.A.words.xml#id(Bdb001.w.691)..id(Bdb001.w.700)
Bdb001.A.words.xml#id(Bdb001.w.701)..id(Bdb001.w.702)
Bdb001.A.words.xml#id(Bdb001.w.902)..id(Bdb001.w.903)
Bdb001.A.words.xml#id(Bdb001.w.915)..id(Bdb001.w.922)
Bdb001.A.words.xml#id(Bdb001.w.929)..id(Bdb001.w.930)
Bdb001.A.words.xml#id(Bdb001.w.1,171)..id(Bdb001.w.1,173)
Bdb001.A.words.xml#id(Bdb001.w.1,251)..id(Bdb001.disfmarker.49)
Bdb001.A.words.xml#id(Bdb001.w.1,255)..id(Bdb001.w.1,261)
Bdb001.A.words.xml#id(Bdb001.w.1,262)..id(Bdb001.w.1,297)
Bdb001.A.words.xml#id(Bdb001.w.1,340)..id(Bdb001.w.1,349)
Bdb001.A.words.xml#id(Bdb001.w.1,350)..id(Bdb001.disfmarker.54)
Bdb001.A.words.xml#id(Bdb001.w.1,353)..id(Bdb001.w.1,361)
Bdb001.A.words.xml#id(Bdb001.w.1,362)..id(Bdb001.disfmarker.56)
Bdb001.A.words.xml#id(Bdb001.w.1,476)
Bdb001.A.words.xml#id(Bdb001.w.1,478)..id(Bdb001.w.1,479)
Bdb001.A.words.xml#id(Bdb001.w.1,480)..id(Bdb001.w.1,490)
Bdb001.A.words.x

### Extract summaries

In [29]:
meeting_summaries = dict() # key = meeting id, value =  ordered dict of word id: word


In [72]:
"Bdb001.A.words.xml#id(Bdb001.w.1,362)..id(Bdb001.disfmarker.56)".split("..")

['Bdb001.A.words.xml#id(Bdb001.w.1,362)', 'id(Bdb001.disfmarker.56)']

In [73]:
"Bdb001.A.words.xml#id(Bdb001.w.1,476)".split("..")

['Bdb001.A.words.xml#id(Bdb001.w.1,476)']